In [12]:
import requests
import json
from pathlib import Path
from typing import Optional

# --- Function to load the token (embedded in this script) ---

def load_token_from_file(filepath: str) -> Optional[str]:
    """
    Loads a token from a file, ignoring comments and empty lines.
    This can handle both relative and absolute file paths.

    Args:
        filepath (str): The path to the file containing the token.

    Returns:
        Optional[str]: The token, or None if the file is not found or is empty.
    """
    token_file = Path(filepath)
    try:
        if token_file.exists():
            with open(token_file, 'r', encoding='utf-8') as f:
                for line in f:
                    line = line.strip()
                    # Skip comments (lines starting with #) and empty lines
                    if line and not line.startswith('#'):
                        return line
        return None
    except Exception as e:
        print(f"❌ Error reading token file {filepath}: {e}")
        return None

# --- Main function to diagnose the token loading process ---

def run_token_diagnostic() -> None:
    """
    DIAGNOSTIC VERSION:
    This function only loads the key from the specified path and prints
    information about it for verification. It does not send any API requests.
    """
    print("🚀 Starting token loading diagnostic...")
    
    # The absolute path to your token file.
    token_path = '/Users/svitlanakovalivska/CNC/LLM_Project/config/ionos_token.txt'
    
    # Attempt to load the API key.
    api_key = load_token_from_file(token_path)
    
    print("\n" + "="*50)
    # Check if the key was loaded successfully.
    if not api_key:
        print("❌ RESULT: Key was NOT found or the file is empty.")
        print(f"   Please check that the path '{token_path}' is correct and the file is not empty.")
    else:
        print("✅ RESULT: Key was successfully loaded from the file.")
        print(f"   - Key Length: {len(api_key)} characters")
        print(f"   - First 15 characters: {api_key[:15]}...")
        print(f"   - Last 15 characters: ...{api_key[-15:]}")
        
        # Check for hidden leading/trailing whitespace.
        if api_key.startswith(' ') or api_key.endswith(' '):
            print("   ⚠️ WARNING: There are leading or trailing spaces in your key!")
        else:
            print("   - No leading/trailing spaces were detected.")
    print("="*50)

# --- This line runs the diagnostic function when you execute the script ---
if __name__ == "__main__":
    run_token_diagnostic()

🚀 Starting token loading diagnostic...

✅ RESULT: Key was successfully loaded from the file.
   - Key Length: 1622 characters
   - First 15 characters: eyJ0eXAiOiJKV1Q...
   - Last 15 characters: ...OiV1KtSYg-3c8bg
   - No leading/trailing spaces were detected.


## ✅ IONOS API - Успішно протестовано!

**Результати curl тестування (як рекомендувала служба підтримки):**
- ✅ Токен працює відмінно
- ✅ API повністю доступне  
- ✅ Chat completion функціональний
- 🤖 Доступно 13 моделей

**Основні доступні моделі для тестування:**
- `meta-llama/Meta-Llama-3.1-8B-Instruct` (швидка)
- `meta-llama/Llama-3.3-70B-Instruct` (потужна) 
- `meta-llama/Meta-Llama-3.1-405B-Instruct-FP8` (найпотужніша)
- `mistralai/Mistral-Nemo-Instruct-2407`
- `mistralai/Mistral-Small-24B-Instruct`

**⚠️ Важливо:** Використовуйте точні назви моделей з великої літери "Meta-Llama"

In [13]:
import requests
import json
from pathlib import Path
from typing import Optional

# --- Function to load the token (embedded in this script) ---

def load_token_from_file(filepath: str) -> Optional[str]:
    """
    Loads a token from a file, ignoring comments and empty lines.
    This can handle both relative and absolute file paths.

    Args:
        filepath (str): The path to the file containing the token.

    Returns:
        Optional[str]: The token, or None if the file is not found or is empty.
    """
    token_file = Path(filepath)
    try:
        if token_file.exists():
            with open(token_file, 'r', encoding='utf-8') as f:
                for line in f:
                    line = line.strip()
                    # Skip comments (lines starting with #) and empty lines
                    if line and not line.startswith('#'):
                        return line
        print(f"⚠️ Token file not found at path: {filepath}")
        return None
    except Exception as e:
        print(f"❌ Error reading token file {filepath}: {e}")
        return None

# --- Main function to query the model and count tokens ---

def get_answer_and_usage_stats() -> None:
    """
    Sends a basic prompt to the IONOS model, gets the response, and fetches token usage statistics.
    """
    print("🚀 Starting a test request to IONOS AI...")
    
    # 1. Absolute path to your token file is specified here.
    token_path = '/Users/svitlanakovalivska/CNC/LLM_Project/config/ionos_token.txt'
    
    # Load the API key
    api_key = load_token_from_file(token_path)
    if not api_key:
        print("❌ Process stopped because the API key could not be loaded.")
        return

    # 2. API Settings
    endpoint = "https://openai.inference.de-txl.ionos.com/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    # 3. Define our test prompt
    #    A simple math question for the model
    test_prompt = "What is 12 multiplied by 12?"
    
    payload = {
        "model": "meta-llama/Meta-Llama-3.1-8B-Instruct", # CORRECT model name from API
        "messages": [
            {
                "role": "system",
                "content": "You are a helpful assistant."
            },
            {
                "role": "user",
                "content": test_prompt
            }
        ],
        "max_tokens": 100,
        "temperature": 0.1
    }

    print(f"💬 Sending prompt to the model: '{test_prompt}'")

    # 4. Send the request and get the response
    try:
        response = requests.post(endpoint, json=payload, headers=headers, timeout=60)
        response.raise_for_status()  # Check for HTTP errors (like 401, 404, 500)

        data = response.json()
        
        # 5. Extract the data from the response
        # The model's answer
        model_answer = data['choices'][0]['message']['content'].strip()
        
        # The token usage statistics
        usage_stats = data.get('usage', {}) # Use .get to avoid an error if 'usage' is missing
        prompt_tokens = usage_stats.get('prompt_tokens', 'N/A')
        completion_tokens = usage_stats.get('completion_tokens', 'N/A')
        total_tokens = usage_stats.get('total_tokens', 'N/A')

        # 6. Display the results
        print("\n" + "="*40)
        print("✅ Response received successfully!")
        print(f"🤖 Model's Answer: {model_answer}")
        print("-" * 20)
        print("📊 Token Usage Statistics:")
        print(f"   - Prompt Tokens: {prompt_tokens}")
        print(f"   - Completion Tokens: {completion_tokens}")
        print(f"   - Total Tokens Used: {total_tokens}")
        print("="*40)

    except requests.exceptions.RequestException as e:
        print(f"❌ API Connection Error: {e}")
    except KeyError as e:
        print(f"❌ Unexpected API response format. Could not find key: {e}")
    except Exception as e:
        print(f"❌ An unknown error occurred: {e}")

# --- Run the main function ---
if __name__ == "__main__":
    get_answer_and_usage_stats()

🚀 Starting a test request to IONOS AI...
💬 Sending prompt to the model: 'What is 12 multiplied by 12?'

✅ Response received successfully!
🤖 Model's Answer: 12 multiplied by 12 is 144.
--------------------
📊 Token Usage Statistics:
   - Prompt Tokens: 50
   - Completion Tokens: 10
   - Total Tokens Used: 60

✅ Response received successfully!
🤖 Model's Answer: 12 multiplied by 12 is 144.
--------------------
📊 Token Usage Statistics:
   - Prompt Tokens: 50
   - Completion Tokens: 10
   - Total Tokens Used: 60
